# Imports

In [32]:
import math
import numpy as np
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.metrics  import accuracy_score, f1_score , precision_score , recall_score

In [33]:
# nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Amir
[nltk_data]     Kashi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
df = pd.read_csv('emails.csv')

In [35]:
df.head(5)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [36]:
data = df.to_numpy()

In [37]:
# sent_tokenize("asdfasdlf . adfaldkfajd. adkfjlals;dfja.adfal;ksdjfaldsjfl adsklfjak;ewrwer ipoias;dflkj .")

### lower_case all

In [38]:
df['text'] = df['text'].apply(lambda x: x.lower())

In [39]:
df.head(5)

,text,spam
0,subject: naturally irresistible your corporate...,1
1,subject: the stock trading gunslinger fanny i...,1
2,subject: unbelievable new homes made easy im ...,1
3,subject: 4 color printing special request add...,1
4,"subject: do not have money , get software cds ...",1


### remove punctuation

In [40]:
df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df

,text,spam
0,subject naturally irresistible your corporate ...,1
1,subject the stock trading gunslinger fanny is...,1
2,subject unbelievable new homes made easy im w...,1
3,subject 4 color printing special request addi...,1
4,subject do not have money get software cds fr...,1
...,...,...
5723,subject re research and development charges t...,0
5724,subject re receipts from visit jim thanks ...,0
5725,subject re enron case study update wow all ...,0
5726,subject re interest david please call shi...,0


### remove numbers

In [41]:
df['text'] = df['text'].apply(lambda x: re.sub(r'\d+', '', x))
df.head(5)

,text,spam
0,subject naturally irresistible your corporate ...,1
1,subject the stock trading gunslinger fanny is...,1
2,subject unbelievable new homes made easy im w...,1
3,subject color printing special request addit...,1
4,subject do not have money get software cds fr...,1


### tokenize

In [42]:
df['text'] = df['text'].apply(lambda x: word_tokenize(x))

In [43]:
df['text']

0       [subject, naturally, irresistible, your, corpo...
1       [subject, the, stock, trading, gunslinger, fan...
2       [subject, unbelievable, new, homes, made, easy...
3       [subject, color, printing, special, request, a...
4       [subject, do, not, have, money, get, software,...
                              ...                        
5723    [subject, re, research, and, development, char...
5724    [subject, re, receipts, from, visit, jim, than...
5725    [subject, re, enron, case, study, update, wow,...
5726    [subject, re, interest, david, please, call, s...
5727    [subject, news, aurora, update, aurora, versio...
Name: text, Length: 5728, dtype: object

### remove stop words

In [44]:
stop_words = set(stopwords.words('english'))

df['text'] = df['text'].apply(lambda x: [w for w in x if not w.lower() in stop_words])

df.head(5)

,text,spam
0,"[subject, naturally, irresistible, corporate, ...",1
1,"[subject, stock, trading, gunslinger, fanny, m...",1
2,"[subject, unbelievable, new, homes, made, easy...",1
3,"[subject, color, printing, special, request, a...",1
4,"[subject, money, get, software, cds, software,...",1


### stemming

In [45]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()

df['text'] = df['text'].apply(lambda x: [ps.stem(w) for w in x])

df.head(5)

,text,spam
0,"[subject, natur, irresist, corpor, ident, lt, ...",1
1,"[subject, stock, trade, gunsling, fanni, merri...",1
2,"[subject, unbeliev, new, home, made, easi, im,...",1
3,"[subject, color, print, special, request, addi...",1
4,"[subject, money, get, softwar, cd, softwar, co...",1


In [64]:
stop_words = set(stopwords.words('english'))

df['text'] = df['text'].apply(lambda x: [w for w in x if  len(w) > 1])

df.head(5)

,text,spam
0,"[subject, natur, irresist, corpor, ident, lt, ...",1
1,"[subject, stock, trade, gunsling, fanni, merri...",1
2,"[subject, unbeliev, new, home, made, easi, im,...",1
3,"[subject, color, print, special, request, addi...",1
4,"[subject, money, get, softwar, cd, softwar, co...",1


# B

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
train, test = train_test_split(df, test_size=0.2)

In [67]:
train = train.to_numpy()
test = test.to_numpy()

In [68]:
def calc_number_of_each_word(matrix: np.ndarray):
    # matrix has two column first is list of word and second is label
    all_words_count = {}
    data_x = []
    data_y = matrix[:, 1]

    words_counts_in_true = {}
    words_counts_in_false = {}


    true_numbers = 0
    false_numbers = 0

    for id in range(matrix.shape[0]):
        msg = matrix[id, 0]
        msg_word_count = {}
        for word in msg:
            msg_word_count[word] = msg_word_count.get(word, 0) + 1
            all_words_count[word] = all_words_count.get(word, 0) + 1

            if matrix[id, 1] == 1:
                words_counts_in_true[word] = words_counts_in_true.get(word, 0) + 1
                true_numbers += 1
            else:
                words_counts_in_false[word] = words_counts_in_false.get(word, 0) + 1
                false_numbers += 1

        data_x.append(msg_word_count)

    return data_x, data_y, all_words_count, words_counts_in_false, words_counts_in_true, false_numbers, true_numbers

In [69]:
train_x, train_y, all_words, words_counts_in_false, words_counts_in_true, false_numbers, true_numbers = calc_number_of_each_word(
    train)

In [70]:
len(all_words)

23043

In [71]:
words_counts_in_true

{'subject': 1258,
 'million': 246,
 'fresh': 47,
 'email': 678,
 'address': 337,
 'sent': 114,
 'cd': 113,
 'free': 497,
 'tri': 104,
 'buy': 167,
 'get': 586,
 'readi': 100,
 'deal': 65,
 'never': 103,
 'seen': 23,
 'hot': 88,
 'much': 152,
 'send': 276,
 'powerman': 3,
 'internet': 213,
 'market': 469,
 'shop': 97,
 'freah': 1,
 'fulli': 23,
 'function': 9,
 'websit': 424,
 'take': 252,
 'order': 388,
 'proven': 66,
 'high': 139,
 'impact': 20,
 'resal': 2,
 'product': 328,
 'ebook': 14,
 'cut': 24,
 'edg': 17,
 'tool': 71,
 'instruct': 64,
 'less': 65,
 'flame': 2,
 'non': 35,
 'buyer': 14,
 'contact': 195,
 'anti': 11,
 'commerc': 10,
 'radic': 1,
 'extremist': 1,
 'rememb': 84,
 'potenti': 103,
 'incom': 87,
 'chart': 11,
 'begin': 21,
 'messag': 344,
 'imagin': 49,
 'kind': 59,
 'money': 542,
 'could': 189,
 'make': 464,
 'mail': 640,
 'one': 470,
 'piec': 8,
 'sold': 30,
 'tenth': 1,
 'percent': 31,
 'math': 1,
 'amaz': 25,
 'prove': 18,
 'best': 288,
 'compar': 39,
 'term': 125

In [72]:
test_x, test_y, _, _, _, _, _ = calc_number_of_each_word(test)

In [73]:
test_x

[{'subject': 5,
  'fmrc': 8,
  'mark': 6,
  'thank': 5,
  'info': 1,
  'shall': 1,
  'check': 1,
  'vinc': 3,
  'courtney': 4,
  'pm': 3,
  'kaminski': 1,
  'hou': 3,
  'ect': 5,
  'cc': 3,
  'joe': 10,
  'gordon': 8,
  'corp': 3,
  'enron': 12,
  'ran': 1,
  'across': 1,
  'recruit': 2,
  'vanderbilt': 4,
  'head': 1,
  'han': 4,
  'stoll': 6,
  'one': 3,
  'lead': 1,
  'deriv': 1,
  'academician': 1,
  'countri': 1,
  'look': 1,
  'websit': 1,
  'briefli': 1,
  'thought': 1,
  'might': 1,
  'interest': 4,
  'also': 2,
  'forward': 3,
  'email': 1,
  'correspond': 1,
  'dr': 3,
  'associ': 1,
  'former': 1,
  'student': 1,
  'owen': 2,
  'school': 1,
  'success': 1,
  'late': 1,
  'undergrad': 1,
  'mba': 2,
  'level': 1,
  'gotten': 1,
  'high': 2,
  'qualiti': 1,
  'peopl': 1,
  'pleas': 1,
  'let': 2,
  'know': 2,
  'would': 6,
  'glad': 1,
  'follow': 1,
  'help': 4,
  'way': 2,
  'north': 1,
  'america': 1,
  'drop': 1,
  'ball': 1,
  'far': 1,
  'get': 2,
  'someon': 3,
  'april

In [81]:
alpha = 1
prob_true = true_numbers/(true_numbers+false_numbers)
prob_false = false_numbers/(true_numbers+false_numbers)

# true is spam
# false is not spam

def calc_naive_bayes(xtest):
    test_words = {}
    for word in xtest:
        test_words[word] = test_words.get(word, 0) + 1

    # calc prob in to be true
    prob_be_true = math.log(prob_true)

    for word in test_words.keys():
        # p = (words_counts_in_true.get(word,0) + alpha) / (true_numbers+ (alpha * true_numbers+false_numbers))
        p = (words_counts_in_true.get(word,0) + 1) / true_numbers
        prob_be_true += math.log(p)


    # calc prob in to be flase
    prob_be_false = math.log(prob_false)

    for word in test_words.keys():
        # p = (words_counts_in_false.get(word,0) + alpha) / (false_numbers+ (alpha * true_numbers+false_numbers))
        p = (words_counts_in_false.get(word,0) + 1) / false_numbers
        prob_be_false += math.log(p)

    if prob_be_true > prob_be_false :
        return int(1)
    else:
        return int(0)


In [82]:
preds = []
for test in test_x:
    res = calc_naive_bayes(test)
    preds.append(res)
preds =np.array(preds)
preds

array([0, 0, 0, ..., 0, 0, 0])

In [83]:
test_y = np.int32(test_y)

# C

In [84]:
accuracy_score(test_y,preds)

0.974694589877836

In [85]:
recall_score(test_y,preds)

0.9927007299270073

In [86]:
precision_score(test_y,preds)

0.9096989966555183

In [87]:
f1_score(test_y,preds)

0.9493891797556719